## Autoreload

Autoreload allows the notebook to dynamically load code: if we update some helper functions *outside* of the notebook, we do not need to reload the notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

In [567]:
# All auxiliary code is in ../src

import sys

sys.path.append("../src/")

# Data understanding

As first thing we imported all necessary modules

In [568]:
import pandas as pd

Then we load our dataset

In [569]:
races = pd.read_csv('../dataset/races.csv', sep=",")

In [ ]:
races.dtypes

Diamo una prima occhiata ai valori mancanti nulli

In [ ]:
races[races.isnull().any(axis=1)]

### Dataframe with only race attributes

In [572]:
cyclist_races_columns = ['position', 'cyclist', 'cyclist_age', 'cyclist_team', 'delta', 'date']
races_columns = [col for col in races.columns if col not in cyclist_races_columns]
races_data = races.drop_duplicates(subset=races_columns)[races_columns].reset_index(drop=True)

Come prima cosa distingurei i valori delle singole corse con i dati relativi ai ciclisti della corsa per cercare di rimuovere ridondanza dei dati che potrebbe falsare le nostre distribuzioni a favore dei valori delle corse con più ciclisti -> separiamo in due tabelle normalizzate

### Histograms Plotting

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


for feature in races_data.select_dtypes(include="number").columns: 
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Istogramma per races_data
    sns.histplot(races_data[feature], ax=axes[0])
    axes[0].set_title(f'Histogram of {feature} in races_data')
    axes[0].set_xlabel(feature)
    axes[0].set_ylabel('Frequency')

    # Istogramma per races
    sns.histplot(races[feature], ax=axes[1])
    axes[1].set_title(f'Histogram of {feature} in races')
    axes[1].set_xlabel(feature)
    axes[1].set_ylabel('Frequency')

    plt.tight_layout()  # Per evitare sovrapposizioni
    plt.show()

# Stats on attributes

In [574]:
from matplotlib import pyplot as plt
import numpy as np


def stats(column, box=False):
    print(f"Description of attribute '{column.name}':")
    display(column.describe())
    print("\nUnique values:")
    print(column.unique())
    print(f"\nNumber of null values: {column.isnull().sum()}")
    mv = column.isna().sum()
    nrec = races.shape[0]
    per=mv*100/nrec
    print(f"\n{mv} null values over {nrec} records - ({per:.2f}%)")
    print("\nTop 5 common value:" + "\n"+str(column.value_counts().head()))
    
   
    if box:
        boxplot_dict = plt.boxplot(column[~np.isnan(column)])
        #recover outliers from boxplot
        outliers = [flier.get_ydata() for flier in boxplot_dict['fliers']]
        #get the list of outliers without duplicates
        outliers_values = list({value for sublist in outliers for value in sublist})
        print("\nOutliers:", outliers_values)

Age

Ci accorgiamo che il 13-enne è un errore perchè le gare ammettono maggiorenni. Il 56-enne è un outliers. Controllare i valori al di fuori della coda del boxplot

In [ ]:
stats(races["cyclist_age"], box=True)

Url

In [ ]:
stats(races["_url"])

Name

In [ ]:
stats(races["name"])

In [ ]:
stats(races["points"], box=True)

In [ ]:
stats(races["uci_points"], box=True)

In [ ]:
stats(races["length"], box=True)

In [ ]:
stats(races["climb_total"], box=True)

In [ ]:
stats(races["uci_points"], box=True)

In [ ]:
stats(races["profile"], box=True) # ? occhio a media su categorico

In [ ]:
stats(races["startlist_quality"], box=True)

In [ ]:
stats(races["average_temperature"], box=True)

In [ ]:
stats(races["date"])

In [ ]:
stats(races["position"]) # ordinale numerico come gestirlo?

In [ ]:
stats(races["cyclist"])

In [ ]:
stats(races["cyclist_age"], box=True)

In [ ]:
stats(races["is_tarmac"])

In [ ]:
stats(races["is_cobbled"])

In [ ]:
stats(races["is_gravel"])

In [ ]:
stats(races["cyclist_team"])

In [ ]:
stats(races["delta"], box=True)

In [ ]:
races[races["_url"].str.startswith('vuelta-a-espana/1996/stage')]

# Data quality

## Age

Il box plot relativo a questo attributo, mostrava la presenza di diversi outliers, in particolare ci siamo focalizzati su due di questi, con valori rispettivamente di 13 e 56.

In [ ]:
races[(races['cyclist_age'] == 13) | (races['cyclist_age'] == 56)]


Considerato che il dataset "cyclists" riporta l'anno di nascita del ciclista "planem-stanev" (1988), e che l'unica gara a cui ha partecipato si è tenuta nel 2001, di conseguenza l'età (13) che presentava in quella gara coincide con quella indicata nel dataset.

Di seguito verifichiamo che ci sia consistenza tra l'anno di nascita indicata nel dataset "cyclists" e l'età durante la partecipazione alla gara indicata nel dataset "races".

## Checking for age consistency detection between the two datasets

In [ ]:
races.head()

In [ ]:
cyclists_df = pd.read_csv('../dataset/cyclists.csv')
cyclists_df.head()


### Get unique cyclists from races dataset

In [599]:
cyclists_races = races['cyclist'].unique()
cyclists_races_count = len(cyclists_races)


### Get unique cyclists from cyclists dataset

In [ ]:
cyclists_uniques = cyclists_df['_url'].unique()
cyclists_unique_count = len(cyclists_uniques)
if (len(cyclists_df['_url']) == cyclists_unique_count):
    print("Correctly each cyclist url appears only one time in the cyclists dataset as row")


### Compare number of cyclists
We can see that races dataset contains 39 more cyclist than cyclists dataset

In [ ]:
print(f"Number of unique cyclists in the cyclists dataset: {cyclists_unique_count}\
      \nNumber of unique cyclists in the races dataset: {cyclists_races_count}")

### Compare cyclists
We show the list of different cyclists between the two datasets

In [ ]:
diff = np.setxor1d(cyclists_uniques, cyclists_races)
print(f"{len(diff)} different cyclists between the two datasets: {diff}")


We'll compare only the age of cyclists that appear in both datasets.
First we'll need to transform the date field of the races datasets, to make comparison easier. We'll transform from object to pandas datetime type

In [ ]:
print(races['date'].dtype)
races['date'] = pd.to_datetime(races['date']).dt.floor('d')


For what concern the field birth_year in the cyclists dataset, we'll transform from float to int, because here we have only the year information

In [ ]:
print(cyclists_df['birth_year'].dtype)
#by using Int64 we don't get error due to the presence of Nan
cyclists_df['birth_year'] = cyclists_df['birth_year'].astype('Int64')

print(cyclists_df['birth_year'].head())
print(cyclists_df['birth_year'].dtype)  

# cyclists_df['birth_year'] = pd.to_datetime(cyclists_df['birth_year']).dt.date


For what concern cyclist_age field in the races dataset, we'll transform from float to int

In [ ]:
print(races['cyclist_age'].dtype)
races['cyclist_age'] = races['cyclist_age'].astype('Int64')

print(races['cyclist_age'].head())
print(races['cyclist_age'].dtype)  

### Merging datasets on the same cyclist

In [ ]:
merged_df = pd.merge(cyclists_df, races, left_on='_url', right_on='cyclist', how='inner')
print(merged_df.shape)
print(merged_df[['birth_year', 'cyclist_age', 'date']])
# write in a temporary file for offline checking
merged_df.to_csv('../../dataset/merged_df_temp.csv', index=False)

### Checking age consistency
We do this by calculating the expected age at the race moment, and then comparing this age with the one indicated in the races dataset.
We do this step for each row, so for each cyclist and all the stages of all races in which he participated.

If even only one among these fields: birth_year, cyclist_age, date, is not valued, we'll skip the corresponding row.

As we can see, we get a truly consistency between the ages indicated into the two dataset

In [619]:
df_filtered = merged_df.dropna(subset=['birth_year', 'date', 'cyclist_age']).copy()

# we regard only on race year
df_filtered['date'] = df_filtered['date'].dt.year

df_filtered['expected_age'] = df_filtered['date'] - df_filtered['birth_year']

df_filtered['consistent_age'] = (df_filtered['cyclist_age'] == df_filtered['expected_age'])

df_filtered['consistent_age'].all()

df_filtered.to_csv('../../dataset/df_for_age_consistency.csv', index=False)


## Name

Alcuni nomi di gare differiscono per singoli caratteri

In [ ]:
unique_names=races['name'].unique()
sorted(unique_names)

## Delta 

- Alcuni delta negativi
- Primi classificati con delta diverso da 0 
- Inconsistenza tra la semantica della colonna e i valori effettivamente indicati

In [ ]:
neg = races[races['delta'] < 0]['delta']
len(neg)

In [ ]:
races[(races['position'] == 1) & (races['delta'] != 0)]

## is_gravel, is_tarmac, is_cobbled


Righe in cui tutti e 3 sono settati su false. Vuol dire che è un missing values?

In [ ]:
races[(races['is_gravel'] == False) & (races['is_tarmac'] == False) & (races['is_cobbled'] == False)]

- Alcune di queste colonne presentano tutti valori false/true -> potrebbe essere inutile

In [ ]:
races[(races['is_gravel'] == True) | (races['is_cobbled'] == True)].shape[0]


## Average Temperature

Questa feature presenta circa il 95% di valori nulli, dunque non determinante ai fini delle nostre analisi. 

In [ ]:
null_values = len(races[races['average_temperature'].isna()])
print((null_values/races.shape[0])*100)

## Points e UCI_points


- Spiegare differenza
- Dire che uci_points presenta la maggior parte di valori nulli, e quindi può essere droppato
  

## Race attributes


Come prima cosa distingurei i valori delle singole corse con i dati relativi ai ciclisti della corsa per cercare di rimuovere ridondanza dei dati che potrebbe falsare le nostre distribuzioni a favore dei valori delle corse con più ciclisti -> separiamo in due tabelle normalizzate

In [ ]:
# count the frequence of each distinct _url
values_count = races_data['_url'].value_counts()
# if a race has a frequency > 1 then this race may represent an inconsistency between the race's attributes
inconsistent_urls = values_count[values_count > 1]
len(inconsistent_urls)

## Length


In [ ]:
temp = races[races['length'] <= 10000]
temp['_url'].value_counts()

Spiegare che abbiamo trovato dalla distribuzione che molte gare hanno una lunghezza breve, e quindi abbiamo cercato di verificare se ci fosse un errore di scala, oppure se ci sono effettivamente gare/tappe di lunghezza molto breve.

Abbiamo riscontrato che le gare di lunghezza più breve, contengono prologue nell'url (e forse stage-1), e ottenuto conferma che esista effettivamente la tappa iniziale chiamata prologo, di una lunghezza dai 1000 ai 10000 circa metri.


In [ ]:
unique_subset = races[races['_url'].str.contains("prologue")][['_url', 'length']].drop_duplicates(subset=['_url'])
unique_subset.describe()

# Correlazione attributi

In [ ]:
numeric_races_data = races_data.select_dtypes(include="number")
numeric_races_data.corr()

La correlazione tra points e uci_points è molto alta, facciamo uno scatterplot

In [ ]:
seaborn.scatterplot(data=races_data, x="points", y="uci_points")